# Setup Azure ML Connection

This notebook helps you:
1. Connect to your Azure ML Workspace
2. Create a compute cluster
3. Test the connection

Run this notebook first to set up your environment.

## 1. Install Required Packages

If you opened this in Codespaces, dependencies should already be installed via `requirements.txt`.

In [ ]:
# Verify installations
import azure.ai.ml
import azure.identity
import mlflow

print(f"Azure ML SDK version: {azure.ai.ml.__version__}")
print(f"MLflow version: {mlflow.__version__}")

## 2. Azure Authentication

First, authenticate with Azure CLI in the terminal:

```bash
az login --use-device-code
az account list --output table
az account set --subscription "<Your-Subscription-ID>"
```

Then run the cell below to verify authentication:

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if authentication works
    credential.get_token("https://management.azure.com/.default")
    print("✓ Authentication successful!")
except Exception as ex:
    print(f"Authentication failed: {ex}")
    print("Please run 'az login --use-device-code' in the terminal")

## 3. Configure Workspace Connection

Update these values with your Azure ML Workspace details:

In [ ]:
# TODO: Update these values with your workspace details
subscription_id = "<your-subscription-id>"
resource_group = "rg-ml-playground"
workspace_name = "mlw-playground"

## 4. Connect to Azure ML Workspace

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Create ML Client
ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name
)

print(f"✓ Connected to workspace: {ml_client.workspace_name}")
print(f"  Resource group: {ml_client.resource_group_name}")
print(f"  Subscription: {ml_client.subscription_id}")

## 5. List Existing Compute Resources

In [ ]:
# List all compute clusters in the workspace
compute_list = ml_client.compute.list()

print("Existing compute resources:")
for compute in compute_list:
    print(f"  - {compute.name} ({compute.type})")

if not list(ml_client.compute.list()):
    print("  (no compute resources found)")

## 6. Create Compute Cluster (if needed)

This creates a small CPU cluster that scales to 0 when not in use (cost-effective!).

In [ ]:
from azure.ai.ml.entities import AmlCompute

compute_name = "cpu-cluster"

# Check if compute already exists
try:
    compute = ml_client.compute.get(compute_name)
    print(f"✓ Compute cluster '{compute_name}' already exists")
except Exception:
    print(f"Creating compute cluster '{compute_name}'...")
    
    compute = AmlCompute(
        name=compute_name,
        type="amlcompute",
        size="STANDARD_DS3_V2",  # 4 cores, 14 GB RAM
        min_instances=0,  # Scale to 0 when not in use
        max_instances=4,
        idle_time_before_scale_down=120,  # 2 minutes
    )
    
    ml_client.compute.begin_create_or_update(compute).result()
    print(f"✓ Compute cluster '{compute_name}' created successfully!")

## 7. Test Connection with Simple Job

Let's submit a simple test job to verify everything works:

In [ ]:
from azure.ai.ml import command

# Create a simple test job
job = command(
    code="../src",  # Path to source code
    command="python -c 'print(\"Hello from Azure ML!\")'"  # Simple Python command
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    compute=compute_name,
    display_name="test-connection",
)

# Submit the job
returned_job = ml_client.jobs.create_or_update(job)

print(f"✓ Test job submitted!")
print(f"  Job name: {returned_job.name}")
print(f"  Status: {returned_job.status}")
print(f"\nView job in Azure ML Studio:")
print(f"  {returned_job.studio_url}")

## 8. Monitor Job Status

In [ ]:
# Stream job logs (optional)
ml_client.jobs.stream(returned_job.name)

## Summary

If all cells ran successfully, you're ready to go! 🎉

Next steps:
- Open `01_iris_training.ipynb` for a complete training example
- Check Azure ML Studio to see your workspace and jobs

## Troubleshooting

**Authentication issues:**
```bash
az login --use-device-code
az account set --subscription "<your-subscription-id>"
```

**Can't find workspace:**
- Verify subscription_id, resource_group, and workspace_name are correct
- Check Azure Portal to confirm workspace exists

**Compute creation fails:**
- Check Azure subscription limits/quotas
- Try a smaller VM size (e.g., STANDARD_DS2_V2)